# Learn Neo4j and Cypher in Notebook

### References

- https://github.com/elena/py2neo-quickstart
- Cypher Fundamentals by Dave

### Install Neo4j Desktop

```
$ cd ~/neo4j
$ ./neo4j-desktop-1.4.1-x86_64.AppImage
```

Open `neo4j` browser, type:
`:play movies`

Run the script to create `Movies` graph database

### [Cypher Styles](https://github.com/opencypher/openCypher/blob/master/docs/style-guide.adoc)
- Case Sensitive
    - Node labels
    - Relationship types
    - Property keys
- Case Insensitive
    - Cypher keywords
    
- Cypher keywords and clauses should be written in all caps and on their own line
- Functions should be written in lower camel case.

In [12]:
from py2neo import Graph,Node,Relationship

In [13]:
from _connstr import CONN_STR

In [14]:
url, username, password = CONN_STR["url"],CONN_STR["user"],CONN_STR["password"]
graph = Graph(url, auth=(username, password))

### Cypher - Intro

In [38]:
query = """
MATCH (p:Person)
RETURN p
LIMIT 10
"""

results = graph.run(query)
type(results)

py2neo.database.work.Cursor

In [34]:
# results is serialized to dataframe
df = results.to_data_frame()
df

In [37]:
query = """
MATCH (p:Person)
RETURN p
LIMIT 10
"""

results = graph.run(query)

# results is serialized to numpy array
results.to_ndarray()

array([[Node('Person', born=1961, name='Aaron Sorkin')],
       [Node('Person', born=1940, name='Al Pacino')],
       [Node('Person', born=1960, name='Annabella Sciorra')],
       [Node('Person', born=1962, name='Anthony Edwards')],
       [Node('Person', born=1976, name='Audrey Tautou')],
       [Node('Person', born=1967, name='Ben Miles')],
       [Node('Person', born=1955, name='Bill Paxton')],
       [Node('Person', born=1953, name='Bill Pullman')],
       [Node('Person', born=1948, name='Billy Crystal')],
       [Node('Person', born=1961, name='Bonnie Hunt')]], dtype=object)

In [39]:
query = """
MATCH (p:Person)
RETURN p
LIMIT 10
"""

results = graph.run(query)

# must re-run query
results.to_table()

p
"(_2:Person {born: 1961, name: 'Aaron Sorkin'})"
"(_3:Person {born: 1940, name: 'Al Pacino'})"
"(_4:Person {born: 1960, name: 'Annabella Sciorra'})"
"(_5:Person {born: 1962, name: 'Anthony Edwards'})"
"(_8:Person {born: 1976, name: 'Audrey Tautou'})"
"(_9:Person {born: 1967, name: 'Ben Miles'})"
"(_11:Person {born: 1955, name: 'Bill Paxton'})"
"(_12:Person {born: 1953, name: 'Bill Pullman'})"
"(_13:Person {born: 1948, name: 'Billy Crystal'})"
"(_14:Person {born: 1961, name: 'Bonnie Hunt'})"


In [47]:
query = """
// read 10 persons
MATCH (p:Person)
RETURN "Person" as Type, p.name as Name, p.born as Birth_Year
LIMIT 10;
"""

df = graph.run(query).to_data_frame()
df.head(5)

,Type,Name,Birth_Year
0,Person,Aaron Sorkin,1961
1,Person,Al Pacino,1940
2,Person,Annabella Sciorra,1960
3,Person,Anthony Edwards,1962
4,Person,Audrey Tautou,1976


### Cypher - MATCH (Read/Query)

In [49]:
query = """
// find actors and their movies
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
RETURN p, r, m
LIMIT 10;
"""

df = graph.run(query).to_data_frame()
df.head(5)

,p,r,m
0,"{'born': 1967, 'name': 'James Marshall'}",{'roles': ['Pfc. Louden Downey']},{'tagline': 'In the heart of the nation's capi...
1,"{'born': 1943, 'name': 'J.T. Walsh'}",{'roles': ['Lt. Col. Matthew Andrew Markinson']},{'tagline': 'In the heart of the nation's capi...
2,"{'born': 1962, 'name': 'Demi Moore'}",{'roles': ['Lt. Cdr. JoAnne Galloway']},{'tagline': 'In the heart of the nation's capi...
3,"{'born': 1968, 'name': 'Cuba Gooding Jr.'}",{'roles': ['Cpl. Carl Hammaker']},{'tagline': 'In the heart of the nation's capi...
4,"{'born': 1966, 'name': 'Kiefer Sutherland'}",{'roles': ['Lt. Jonathan Kendrick']},{'tagline': 'In the heart of the nation's capi...


In [50]:
df.iloc[0]

p             {'born': 1967, 'name': 'James Marshall'}
r                    {'roles': ['Pfc. Louden Downey']}
m    {'tagline': 'In the heart of the nation's capi...
Name: 0, dtype: object

In [52]:
df.iloc[0].p["name"], df.iloc[0].r["roles"], df.iloc[0].m["title"]  

('James Marshall', ['Pfc. Louden Downey'], 'A Few Good Men')

In [55]:
query = """
// variable
MATCH p = (:Person)-[:ACTED_IN]->(:Movie)
RETURN p
LIMIT 2;
"""

graph.run(query).data()

[{'p': Path(Node('Person', born=1967, name='James Marshall'), ACTED_IN(Node('Person', born=1967, name='James Marshall'), Node('Movie', released=1992, tagline="In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.", title='A Few Good Men')))},
 {'p': Path(Node('Person', born=1943, name='J.T. Walsh'), ACTED_IN(Node('Person', born=1943, name='J.T. Walsh'), Node('Movie', released=1992, tagline="In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.", title='A Few Good Men')))}]

In [65]:
query = """
//WHERE
MATCH (p:Person)
WHERE p.name = 'Tom Hanks'
RETURN p;
"""

graph.run(query).data()

[{'p': Node('Person', born=1956, name='Tom Hanks')}]

In [66]:
query = """
// WHERE with pattern
MATCH (p:Person)-[:WROTE]->(m:Movie)
WHERE (p)-[:PRODUCED]->(m)
RETURN p.name, m.title
LIMIT 3;
"""

graph.run(query).data()

[{'p.name': 'Cameron Crowe', 'm.title': 'Jerry Maguire'},
 {'p.name': 'Nora Ephron', 'm.title': 'When Harry Met Sally'},
 {'p.name': 'Lana Wachowski', 'm.title': 'V for Vendetta'}]

In [67]:
query = """
// WHERE with pattern
MATCH (p:Person)-[:WROTE]->(m:Movie)
WHERE NOT (p)-[:PRODUCED]->(m)
RETURN p.name, m.title
LIMIT 3;
"""

graph.run(query).data()

[{'p.name': 'Aaron Sorkin', 'm.title': 'A Few Good Men'},
 {'p.name': 'Jim Cash', 'm.title': 'Top Gun'},
 {'p.name': 'David Mitchell', 'm.title': 'Cloud Atlas'}]

In [68]:
query = """
MATCH (p:Person)
WHERE NOT (p)-[:ACTED_IN|:DIRECTED]->(:Movie)
RETURN p
LIMIT 3;
"""

graph.run(query).data()

[{'p': Node('Person', born=1952, name='Joel Silver')},
 {'p': Node('Person', born=1941, name='Jim Cash')},
 {'p': Node('Person', born=1969, name='David Mitchell')}]

In [73]:
query = """
MATCH (m:Movie)
WHERE m.title CONTAINS 'Matrix'
RETURN m.title
LIMIT 3;
"""

graph.run(query).data()

[{'m.title': 'The Matrix'},
 {'m.title': 'The Matrix Reloaded'},
 {'m.title': 'The Matrix Revolutions'}]

In [70]:
query = """
// string functions
MATCH (m:Movie)
WHERE m.title STARTS WITH 'The Matrix'
RETURN m.title
LIMIT 4;
"""

graph.run(query).data()

[{'m.title': 'The Matrix'},
 {'m.title': 'The Matrix Reloaded'},
 {'m.title': 'The Matrix Revolutions'}]

In [74]:
query = """
// string functions
MATCH (m:Movie)
WHERE m.title ENDS WITH 'Seattle'
RETURN m.title
LIMIT 4;
"""

graph.run(query).data()

[{'m.title': 'Sleepless in Seattle'}]

In [72]:
query = """
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name STARTS WITH 'Tom'
RETURN p, m
LIMIT 4;
"""

graph.run(query).data()

[{'p': Node('Person', born=1962, name='Tom Cruise'),
  'm': Node('Movie', released=2000, tagline='The rest of his life begins now.', title='Jerry Maguire')},
 {'p': Node('Person', born=1962, name='Tom Cruise'),
  'm': Node('Movie', released=1986, tagline='I feel the need, the need for speed.', title='Top Gun')},
 {'p': Node('Person', born=1962, name='Tom Cruise'),
  'm': Node('Movie', released=1992, tagline="In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.", title='A Few Good Men')},
 {'p': Node('Person', born=1933, name='Tom Skerritt'),
  'm': Node('Movie', released=1986, tagline='I feel the need, the need for speed.', title='Top Gun')}]

In [77]:
query = """
// regex
MATCH (m:Movie)
WHERE m.title =~ '.*[0-9]+.*'
RETURN m.title
LIMIT 3;
"""

graph.run(query).data()

[{'m.title': 'Apollo 13'}]

In [78]:
query = """
// aggregate, order by, limit
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
RETURN p.name, count(*) AS movies
ORDER BY movies DESC
LIMIT 5;
"""

graph.run(query).data()

[{'p.name': 'Tom Hanks', 'movies': 12},
 {'p.name': 'Keanu Reeves', 'movies': 7},
 {'p.name': 'Jack Nicholson', 'movies': 5},
 {'p.name': 'Hugo Weaving', 'movies': 5},
 {'p.name': 'Meg Ryan', 'movies': 5}]

In [79]:
query = """
// multiple match
MATCH (p:Person)-[:ACTED_IN]->(m:Movie),
      (other:Person)-[:ACTED_IN]->(m)
WHERE p.name = 'Meg Ryan'
RETURN other.name, m.title
ORDER BY other.name
LIMIT 3;
"""

graph.run(query).data()

[{'other.name': 'Anthony Edwards', 'm.title': 'Top Gun'},
 {'other.name': 'Bill Pullman', 'm.title': 'Sleepless in Seattle'},
 {'other.name': 'Billy Crystal', 'm.title': 'When Harry Met Sally'}]

In [80]:
query = """
// multiple match
MATCH (p:Person)-[:ACTED_IN]->(m:Movie),
      (other:Person)-[:ACTED_IN]->(m),
      (director:Person)-[:DIRECTED]->(m)
WHERE p.name = 'Meg Ryan'
RETURN m.title AS movie, 
       director.name AS director,
       other.name AS coActor
LIMIT 3;
"""

graph.run(query).data()

[{'movie': 'Sleepless in Seattle',
  'director': 'Nora Ephron',
  'coActor': 'Victor Garber'},
 {'movie': 'Sleepless in Seattle',
  'director': 'Nora Ephron',
  'coActor': 'Tom Hanks'},
 {'movie': 'Sleepless in Seattle',
  'director': 'Nora Ephron',
  'coActor': 'Bill Pullman'}]

In [81]:
query = """
// OPTINOAL MATCH
MATCH (p:Person)
WHERE p.name STARTS WITH 'Tom'
OPTIONAL MATCH (p)-[:DIRECTED]->(m:Movie)
RETURN p.name, m.title
LIMIT 5;
"""

graph.run(query).data()

[{'p.name': 'Tom Cruise', 'm.title': None},
 {'p.name': 'Tom Skerritt', 'm.title': None},
 {'p.name': 'Tom Hanks', 'm.title': 'That Thing You Do'},
 {'p.name': 'Tom Tykwer', 'm.title': 'Cloud Atlas'}]

### Cypher - DELETE

In [62]:
query = """
MATCH (p:Person)
WHERE p.name = 'Emil Eifrem'
RETURN p;
"""

graph.run(query)

 p                                             
-----------------------------------------------
 (_8:Person {born: 1978, name: 'Emil Eifrem'}) 

In [64]:
query = """
// delete one node and relationships
MATCH (p:Person)
WHERE p.name = 'Emil Eifrem'
DETACH DELETE p;
"""

graph.run(query)

(No data)

In [57]:
query = """
//Delete all Movie and Person nodes, and their relationships
MATCH (n) 
DETACH DELETE n;
"""

graph.run(query)

(No data)

In [60]:
query = """
//Prove that the Movie Graph is gone
MATCH (n) RETURN count(*);
"""

graph.run(query)

 count(*) 
----------
      171 